# Importation des packages

In [1]:
from loky import ProcessPoolExecutor
import os
import requests
from bs4 import BeautifulSoup
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import re
from itertools import chain

# Définition de fonctions

### Les "mapper" pour récupérer les `id_titles`

In [2]:
def mapper_id_titles(url):
    response = requests.get(url)
    BS = (BeautifulSoup(response.text)
          .select("html body main div#filterContainer.container.pt-2 div.row div.col-sm-6.col-lg-3.mb-3 div.poster-container a.poster-link"))
    return [i.get('href') for i in BS]

In [3]:
# mapper_id_titles('https://fr.flixable.com/genre/{}/?min-rating=0&min-year=1920&max-year=2019&order=title&page=1')

### Parallélisation pour récupérer les `id_titles`

In [4]:
def all_id_title(key):
    with ProcessPoolExecutor() as epool:
        mapped_values = epool.map(mapper_id_titles, key)
    return list(chain(*mapped_values))

### Les "mapper" sur le site `flixable.com`

In [5]:
def mapper_flixable(id_title):
    flixable = "https://fr.flixable.com" + id_title
    response = requests.get(flixable)
    
    nom = (BeautifulSoup(response.text)
           .select("html body main div.container.mt-4 div.row.mb-3 div.col-12 h1.mb-3")[0]
           .text.split("\n")[1].strip())
    
    inter = (BeautifulSoup(response.text)
             .select("div[class='col-md-8 mb-3'] span[class=imdbRatingPlugin] a"))
    if len(inter):
        lien_IMDb = inter[0]['href']
    else:
        lien_IMDb = ""
        
    inter = (BeautifulSoup(response.text)
             .select("html body main div.container.mt-4 div.row.mb-3 div.col-md-8.mb-3 div.mb-2 span a"))
    inter = ["".join(re.findall(r'.*/actor/.*',str(i))) for i in inter]
    inter = [i for i in inter if i!=""]
    casting = "|#|".join(["".join(re.findall(r'.*>(.*)</a>',i))for i in inter])
    
    date_ajout = (BeautifulSoup(response.text)
                  .select("div.mb-4 > span:nth-child(2)"))[0].text
    
    return (nom, lien_IMDb, casting, date_ajout)

In [6]:
# mapper_flixable("/title/70301585/")

### Parallélisation pour récupérer les données sur le site `flixable.com`

In [7]:
def all_flixable(key):
    with ProcessPoolExecutor() as epool:
        mapped_values_flixable = epool.map(mapper_flixable, key)
    return list(mapped_values_flixable)

### Les "mapper" sur le site `netflix.com/fr`

In [8]:
def mapper_netflix(id_title):
    netflix = "https://www.netflix.com/fr" + id_title
    response = requests.get(netflix)
    
    inter = (BeautifulSoup(response.text)
             .select("a.title-info-metadata-item"))
    genre1 = [i.text.strip().strip(',').replace('\xa0', ' ') for i in inter]
    
    inter = (BeautifulSoup(response.text)
             .select(".cell-genres > div:nth-child(2) > span"))
    genre2 = [i.text.strip().strip(',').replace('\xa0', ' ') for i in inter]
    
    genre = "|#|".join(set(genre1+genre2))
    
    test = BeautifulSoup(response.text).select("#section-more-details")[0].text
    if not re.search(r'.*Ce (?:programme|film) est\.\.\..*', test):
        tag = ""
    else:
        inter = BeautifulSoup(response.text).select("div.more-details-cell div span[class$=tag]")
        tag = "|#|".join([i.text.strip().strip(',').replace('\xa0', ' ') for i in inter])
        
    inter = BeautifulSoup(response.text).select(".link-container a")
    relative = "|#|".join([i['href'].strip('/fr') for i in inter])
    
    return (genre, tag, relative)

In [9]:
# mapper_netflix("/title/70301585/")

### Parallélisation pour récupérer les données sur `netflix.com/fr`

In [10]:
def all_netflix(key):
    with ProcessPoolExecutor() as epool:
        mapped_values_netflix = epool.map(mapper_netflix, key)
    return list(mapped_values_netflix)

### Les "mapper" sur le site `imdb.com`

In [11]:
def mapper_rating_IMDb(id_IMDb):
    if id_IMDb=="":
        return ""
    else:
        response = requests.get(id_IMDb)
        BS = (BeautifulSoup(response.text)
              .select(".ratingValue > strong:nth-child(1) > span:nth-child(1)")[0])
    return float(BS.text)

In [12]:
# mapper_rating_IMDb("https://www.imdb.com/title/tt6213294/?ref_=plg_rt_1")

### Parallélisation pour récupérer les données sur `imdb.com`

In [13]:
def all_rating_IMDb(key):
    with ProcessPoolExecutor() as epool:
        mapped_values_rating_IMDb = epool.map(mapper_rating_IMDb, key)
    return list(mapped_values_rating_IMDb)

# Récupération des données

### Récupération du nombre de pages sur le site `flixable.com`

In [14]:
url = "https://fr.flixable.com/genre/{}/?min-rating=0&min-year=1920&max-year=2020&order=title#filterContainer"
types = ["films", "series-tv"]
number_of_pages = {}

for elem in types:
    response = requests.get(url.format(elem))
    BS = BeautifulSoup(response.text).select("li.page-item:nth-child(7) > a:nth-child(1)")[0]
    number_of_pages[elem] = int(BS.get_text())

number_of_pages

{'films': 64, 'series-tv': 36}

### Récupération des urls du site `flixable.com`

In [15]:
url = "https://fr.flixable.com/genre/{}/?min-rating=0&min-year=1920&max-year=2019&order=title&page="
urls = {}

for elem in types:
    N = number_of_pages[elem]
    urls[elem] = [url.format(elem) + str(i+1) for i in range(N)]

In [16]:
for key, value in urls.items():
    print("{} : {}\n".format(key,value[:2]))

films : ['https://fr.flixable.com/genre/films/?min-rating=0&min-year=1920&max-year=2019&order=title&page=1', 'https://fr.flixable.com/genre/films/?min-rating=0&min-year=1920&max-year=2019&order=title&page=2']

series-tv : ['https://fr.flixable.com/genre/series-tv/?min-rating=0&min-year=1920&max-year=2019&order=title&page=1', 'https://fr.flixable.com/genre/series-tv/?min-rating=0&min-year=1920&max-year=2019&order=title&page=2']



### Récupération des `id_titles`

In [17]:
%%time
id_titles = {}
for key in types:
    id_titles[key] = all_id_title(urls[key])
    
#(~10s)

Wall time: 15.4 s


In [18]:
for key, value in id_titles.items():
    print("Pour les {}, ils y en a {}\n".format(key,len(value)))

Pour les films, ils y en a 2539

Pour les series-tv, ils y en a 1428



In [19]:
for key, value in id_titles.items():
    print("{} : {}\n".format(key,value[:2]))

films : ['/title/80997613/', '/title/70301585/']

series-tv : ['/title/80234414/', '/title/80029823/']



In [20]:
urls_films = id_titles['films']
urls_series = id_titles['series-tv']
numero_titres = urls_films + urls_series

### Récupération des données sur le site `flixable.com`

In [21]:
%%time
data_flixable = {}
for key in types:
    data_flixable[key] = all_flixable(id_titles[key])
    
#(~6min10)

Wall time: 8min 35s


In [22]:
for key, value in data_flixable.items():
    print("{} : {}\n".format(key,value[:2]))

films : [('¡Ay, mi madre!', 'https://www.imdb.com/title/tt6213294/?ref_=plg_rt_1', 'Estefanía de los Santos|#|Secun de la Rosa|#|Terele Pávez|#|María Alfonsa Rosso|#|Mariola Fuentes|#|Alfonso Sánchez|#|Paz Vega|#|Marta Torné|#|Concha Galán', '19/7/2019'), ("'71", 'https://www.imdb.com/title/tt2614684/?ref_=plg_rt_1', "Charlie Murphy|#|Paul Anderson|#|Sam Reid|#|Richard Dormer|#|Sean Harris|#|Killian Scott|#|Martin McCann|#|Corey McKinley|#|David Wilmot|#|Jack O'Connell|#|Charlie Murphy|#|Paul W.S. Anderson|#|Sam Hazeldine", '1/12/2018')]

series-tv : [('100% Hotter', 'https://www.imdb.com/title/tt5819414/?ref_=plg_rt_1', 'Daniel Palmer|#|Melissa Sophia|#|Grace Woodward|#|Karen Williams', '1/8/2019'), ('12 Monkeys', 'https://www.imdb.com/title/tt3148266/?ref_=plg_rt_1', 'Aaron Stanford|#|Amanda Schull|#|Kirk Acevedo|#|Noah Bean|#|Emily Hampshire|#|Barbara Sukowa|#|Todd Stashwick|#|Tom Noonan|#|Zeljko Ivanek', '1/10/2019')]



In [36]:
infos_flixable = []

for elem in data_flixable.values():
    infos_flixable += elem

In [37]:
title = []
urls_IMDb = []
casting = []
date_ajout = []

for elem in infos_flixable:
    title.append(elem[0])
    urls_IMDb.append(elem[1])
    casting.append(elem[2])
    date_ajout.append(elem[3])

In [42]:
types_media = ['film']*len(urls_films) + ['série']*len(urls_series)
types_media[:5],types_media[-5:]

(['film', 'film', 'film', 'film', 'film'],
 ['série', 'série', 'série', 'série', 'série'])

### Récupération des données sur le site `netflix.com/fr`

Ne fonctionne pas car trop de requests en même temps donc Netflix bloque
```
%%time
data_netflix = {}
for key in types:
    data_netflix[key] = all_netflix(id_titles[key])
```

In [ ]:
%%time
infos_netflix = []
URLS = urls_films + urls_series

for elem in URLS:
    infos_netflix.append(mapper_netflix(elem))

In [37]:
genres = []
tags = []
relative = []

for elem in infos_flixable:
    genres.append(elem[0])
    tags.append(elem[1])
    relative.append(elem[2])

### Récupération des données sur le site `imdb.com`

In [23]:
%%time
ratings = all_rating_IMDb(urls_IMDb)
ratings[:10]
#(~3min5)

IndexError: list index out of range

# Création du dataframe

In [11]:
data = {"Titre": title, "Genres Netflix" : genres,
        "Tags Netflix" : tags, "Casting" : casting,
        "Note IMDb" : ratings, "Films/Séries liés" : relative,
        "Date d'ajout" : date_ajout, "Lien Netflix" : numero_titres}

BDD_Netflix = pd.DataFrame(data)
BDD_Netflix

,Titre,Genres Netflix,Genres Flixable,Tags Netflix,Lien Netflix
0,"¡Ay, mi madre!",Films espagnols|#|Comédies,Comédie|#|International,Sentimental,/title/80997613/
1,'71,Films british|#|Action et aventure|#|Action|#|...,Action|#|Drame|#|Indépendant,Violent,/title/70301585/
2,#Horror,Films d'horreur|#|Films indépendants,Horreur|#|Indépendant|#|Thriller,Suspense,/title/80074904/
3,#realityhigh,Films pour ados|#|Comédies,Comédie,,/title/80125979/
4,#Rucker50,Documentaires|#|Docus société et culture|#|Doc...,Documentaires,,/title/80147908/
5,#Selfie,Films pour ados|#|Films roumains|#|Comédies|#|...,Comédie|#|Drame|#|International,,/title/81092768/
6,#Selfie 69,Films roumains|#|Comédies|#|Comédies romantiqu...,Comédie|#|Drame|#|International,Romantique,/title/81092766/
7,10 000,Action et aventure|#|Action|#|Aventure,Action,Montée d'adrénaline|#|Palpitant,/title/70060018/
8,10 Years,Comédies|#|Comédies romantiques|#|Drames|#|Dra...,Comédie|#|Drame|#|Indépendant,Romantique,/title/70212146/
9,100 mètres,Films inspirés de faits réels|#|Films espagnol...,Drame|#|International,,/title/80141173/


In [ ]:
def string_to_datetime(valeur):
    return datetime.strptime(valeur, '%d/%m/%Y')

BDD_Netflix["Date d'ajout"] = BDD_Netflix["Date d'ajout"].apply(string_to_datetime)       

In [ ]:
def list_to_string(valeur):
    if valeur=="":
        return np.nan
    else:
        return valeur

BDD_Netflix["Tags Netflix"] = BDD_Netflix["Tags Netflix"].apply(list_to_string)
BDD_Netflix["Note IMDb"] = BDD_Netflix["Note IMDb"].apply(list_to_string)

# Exportation en `csv` et `pickle`

In [12]:
BDD_Netflix.to_csv('BDD Netflix.csv', sep = ';', index = False, encoding = 'utf-8', na_rep = 'NA')

In [13]:
BDD_Netflix.to_pickle('BDD Netflix.pkl')

***Notre base de données s'arrête au 24 novembre 2019.***

# Sources

- https://fr.flixable.com/?min-rating=0&min-year=1920&max-year=2020&order=title#filterContainer


- https://www.netflix.com/fr/


- http://akul.me/blog/2016/beautifulsoup-cheatsheet/